In [91]:
import json
import pandas as pd
import numpy as np
import cv2
import copy
from ultralytics import YOLO
import shutil
import os
import ast

In [92]:
with open('data/basketball-instants-dataset.json', 'r') as f:
    data = json.load(f)

In [93]:
# extract relevant player position data from dataset json into a cleaner json file

player_data = []
for timestamp in data:
    if timestamp['annotated_human_masks']:
        left_player_timestamp = copy.deepcopy(timestamp)
        right_player_timestamp = copy.deepcopy(timestamp)
        left_player_timestamp['side'] = 'left'
        right_player_timestamp['side'] = 'right'
        left_player_timestamp['annotations'] = []
        right_player_timestamp['annotations'] = []
        for annotation in timestamp['annotations']:
            if annotation['type'] == 'player' and annotation['image'] == 0:
                left_player_timestamp['annotations'].append({
                    "type": "player",
                    "xyz": annotation['head'],
                    "image": annotation['image']
                })
            elif annotation['type'] == 'player' and annotation['image'] == 1:
                right_player_timestamp['annotations'].append({
                    "type": "player",
                    "xyz": [2800 - annotation['head'][0], annotation['head'][1], annotation['head'][2]],
                    "image": annotation['image']
                })
        if len(left_player_timestamp['annotations']) > 0:
            player_data.append(left_player_timestamp)
        if len(right_player_timestamp['annotations']) > 0:
            player_data.append(right_player_timestamp)

In [115]:
# extract relevant ball data from dataset json into a cleaner json file

basketball_data = []

for timestamp in data:
    ball_timestamp = copy.deepcopy(timestamp)
    ball_timestamp['annotations'] = []
    basketball_data.append(ball_timestamp)
    for annotation in timestamp['annotations']:
        if annotation['type'] == 'ball':
            ball_timestamp['annotations'].append(annotation)

In [94]:
with open('data/basketball-positions.json', "w") as f:
    json.dump(basketball_data, f)
with open('data/player-positions.json', "w") as f:
    json.dump(player_data, f)

In [88]:
with open('data/basketball-positions.json', 'r') as f:
    basketball_data = json.load(f)
with open('data/player-positions.json', 'r') as f:
    player_data = json.load(f)

In [90]:
count_with_ball = 0
count_with_player = 0
for timestamp in basketball_data:
    if timestamp['annotations'][0]['visible']:
        count_with_ball += 1
for timestamp in player_data:
    if len(timestamp['annotations']) > 0:
        count_with_player += 1
print("Number of images with ball visible: " + str(count_with_ball))
print("Number of images total: " + str(len(basketball_data)))
print("Number of images with players annotated: " + str(count_with_player))

Number of images with ball visible: 320
Number of images total: 364
Number of images with players annotated: 288


In [4]:
for frame in basketball_data:
    if frame['annotations'][0]['visible']:
        filename1 = 'camcourt1_' + str(frame['timestamp']) + '_' + '0.png'
        filename2 = 'camcourt1_' + str(frame['timestamp']) + '_' + '40.png'
        filename3 = 'camcourt2_' + str(frame['timestamp']) + '_' + '0.png'
        filename4 = 'camcourt2_' + str(frame['timestamp']) + '_' + '40.png'
        shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename1, os.path.join('datasets/yolo-basketball-data/images/', filename1))
        try:
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename2, os.path.join('datasets/yolo-basketball-data/images/', filename2))
        except FileNotFoundError:
            filename2 = filename2[:-6] + '80.png'
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename2, os.path.join('datasets/yolo-basketball-data/images/', filename2))
        shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename3, os.path.join('datasets/yolo-basketball-data/images/', filename3))
        try:
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename4, os.path.join('datasets/yolo-basketball-data/images/', filename4))
        except FileNotFoundError:
            filename4 = filename4[:-6] + '80.png'
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename4, os.path.join('datasets/yolo-basketball-data/images/', filename4))

In [2]:
def str_to_dict(s):
    try:
        return ast.literal_eval(s)
    except ValueError:
        print("ValueError occurred")
        return None

In [8]:
ball_annotations = pd.read_csv('basketball_annotations.csv', converters={'region_shape_attributes': str_to_dict})
ball_annotations = ball_annotations[['filename', 'region_count', 'region_shape_attributes']]
ball_annotations = ball_annotations[ball_annotations['region_count'] == 1]
ball_annotations['is_training'] = np.concatenate([np.array([True] * int(0.8 * len(ball_annotations))), np.array([False] * (len(ball_annotations) - int(0.8 * len(ball_annotations))))])
ball_annotations['is_training'] = ball_annotations['is_training'].sample(frac=1).reset_index(drop=True)

In [9]:
ball_annotations.head()

,filename,region_count,region_shape_attributes,is_training
0,camcourt1_1512405572696_0.png,1,"{'name': 'rect', 'x': 1415, 'y': 1090, 'width'...",True
1,camcourt1_1512405572696_40.png,1,"{'name': 'rect', 'x': 1342, 'y': 1087, 'width'...",True
2,camcourt1_1512405932787_0.png,1,"{'name': 'rect', 'x': 535, 'y': 1092, 'width':...",True
3,camcourt1_1512405932787_40.png,1,"{'name': 'rect', 'x': 524, 'y': 1090, 'width':...",False
16,camcourt1_1512421952112_0.png,1,"{'name': 'rect', 'x': 1427, 'y': 921, 'width':...",False


In [47]:
for index, row in ball_annotations.iterrows():
    label_filepath = ""
    is_train = row['is_training']
    filename = row['filename']
    label_filename = filename[:-4] + ".txt"
    annotation = row['region_shape_attributes']
    height, width, _ = cv2.imread('datasets/yolo-basketball-data/images/' + filename).shape
    if is_train:
        label_filepath = os.path.join('datasets/yolo-basketball-data/train/labels', label_filename)
        shutil.move('datasets/yolo-basketball-data/images/' + filename, 'datasets/yolo-basketball-data/train/images')
    else:
        label_filepath = os.path.join('datasets/yolo-basketball-data/val/labels', label_filename)
        shutil.move('datasets/yolo-basketball-data/images/' + filename, 'datasets/yolo-basketball-data/val/images')
    normal_x = (annotation['x'] + annotation['width']/2) / width
    normal_y = (annotation['y'] + annotation['height']/2) / height
    normal_width = annotation['width'] / width
    normal_height = annotation['height'] / height
    with open(label_filepath, 'w') as f:
        f.write(f"0 {normal_x} {normal_y} {normal_width} {normal_height}")

In [3]:
court_annotations = pd.read_csv('court_lines_annotation_csv.csv', converters={'region_shape_attributes': str_to_dict})
court_annotations = court_annotations[['filename', 'region_count', 'region_shape_attributes']]
court_annotations = court_annotations[court_annotations['region_count'] >= 1]
court_annotations['is_training'] = np.concatenate([np.array([True] * int(0.8 * len(court_annotations))), np.array([False] * (len(court_annotations) - int(0.8 * len(court_annotations))))])
court_annotations['is_training'] = court_annotations['is_training'].sample(frac=1).reset_index(drop=True)

In [4]:
court_annotations.head()

,filename,region_count,region_shape_attributes,is_training
0,camcourt1_1579292188102_0.png,1,"{'name': 'polygon', 'all_points_x': [130, 1040...",True
1,camcourt1_1579292188102_40.png,1,"{'name': 'polygon', 'all_points_x': [130, 1040...",True
2,camcourt1_1579966753552_0.png,1,"{'name': 'polygon', 'all_points_x': [130, 1043...",True
3,camcourt1_1579966753552_40.png,1,"{'name': 'polygon', 'all_points_x': [128, 1043...",True
4,camcourt1_1579966954351_0.png,1,"{'name': 'polygon', 'all_points_x': [130, 1038...",False


In [8]:
for index, row in court_annotations.iterrows():
    label_filepath = ""
    is_train = row['is_training']
    filename = row['filename']
    label_filename = filename[:-4] + ".txt"
    annotation = row['region_shape_attributes']
    height, width, _ = cv2.imread('datasets/court-lines-data/' + filename).shape
    if is_train:
        label_filepath = os.path.join('datasets/court-lines-data/train/labels', label_filename)
        shutil.move('datasets/court-lines-data/' + filename, 'datasets/court-lines-data/train/images')
    else:
        label_filepath = os.path.join('datasets/court-lines-data/val/labels', label_filename)
        shutil.move('datasets/court-lines-data/' + filename, 'datasets/court-lines-data/val/images')
    normal_x1 = annotation['all_points_x'][0] / width
    normal_y1 = annotation['all_points_y'][0] / height
    normal_x2 = annotation['all_points_x'][1] / width
    normal_y2 = annotation['all_points_y'][1] / height
    normal_x3 = annotation['all_points_x'][2] / width
    normal_y3 = annotation['all_points_y'][2] / height
    normal_x4 = annotation['all_points_x'][3] / width
    normal_y4 = annotation['all_points_y'][3] / height
    with open(label_filepath, 'w') as f:
        f.write(f"0 {normal_x1} {normal_y1} {normal_x2} {normal_y2} {normal_x3} {normal_y3} {normal_x4} {normal_y4}")